In [2]:
import pandas as pd
import keras
from keras.models import load_model
import numpy as np
from tqdm import tqdm

Using TensorFlow backend.


In [3]:
model_path = './models/try_2_features.h5'

df = pd.read_csv('./data/csv/test_acc10_2_300.csv')
model = load_model(model_path)

In [11]:
# group by ligand
# for each group, compute top10 accuracy manually

matches = 0
total = float(len(df.lig_id.unique()))
debug = False

for lig_id, grp in tqdm(df.groupby('lig_id')):
    grp.reset_index(inplace=True)
    
    match_id = grp.index[grp['pro_id']==lig_id]
    
    # load all features
    probabilities = []
    n_channels = 4
    dims = (24,24,24)
    X = np.empty((len(grp), *dims, n_channels))
    for row in grp.itertuples():
        X[row[0],] = np.load(row.dest)
    
    probs = model.predict(X, batch_size=200)
    probs = probs.flatten()
    
    largest_first = list(reversed(np.argsort(probs).tolist()))
    top10 = largest_first[:10]
    
    if debug:
        print('ligid: ', lig_id)
        print('match_id: ', match_id)
        print('probs: ', probs)
        print('top_10: ', top10)
        print()
        break
    
    if match_id in top10:
        matches += 1




  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 1/300 [00:03<16:47,  3.37s/it]

  1%|          | 2/300 [00:04<13:36,  2.74s/it]

  1%|          | 3/300 [00:05<11:18,  2.28s/it]

  1%|▏         | 4/300 [00:07<09:41,  1.97s/it]

  2%|▏         | 5/300 [00:08<08:33,  1.74s/it]

  2%|▏         | 6/300 [00:09<07:45,  1.58s/it]

  2%|▏         | 7/300 [00:10<07:12,  1.48s/it]

  3%|▎         | 8/300 [00:11<06:48,  1.40s/it]

  3%|▎         | 9/300 [00:13<06:31,  1.35s/it]

  3%|▎         | 10/300 [00:14<06:19,  1.31s/it]

  4%|▎         | 11/300 [00:15<06:11,  1.29s/it]

  4%|▍         | 12/300 [00:16<06:05,  1.27s/it]

  4%|▍         | 13/300 [00:18<06:00,  1.26s/it]

  5%|▍         | 14/300 [00:19<05:57,  1.25s/it]

  5%|▌         | 15/300 [00:20<05:54,  1.24s/it]

  5%|▌         | 16/300 [00:21<05:51,  1.24s/it]

  6%|▌         | 17/300 [00:23<05:49,  1.23s/it]

  6%|▌         | 18/300 [00:24<05:47,  1.23s/it]

  6%|▋         | 19/300 [00:25<05:46,  1.23s/it]

  7%|▋         |

In [12]:
print(matches/total)    

0.9666666666666667


In [13]:
print(matches/total)    

0.9666666666666667
